# Machine Learning

In [1]:
import sqlite3
import pandas as pd
import numpy as np

conn = sqlite3.Connection("./capstone.sqlite")

df = pd.read_sql("SELECT * FROM open_pv_clean", con=conn)

In [2]:
import matplotlib.pyplot as ply
import seaborn as sns
import plotly.plotly as py

## Incentive Programs

I'd like to get down the number of incentive program names, similar to utility names.

In [3]:
# it's not actually too bad, but maybe there's some mislabeling?

len(df.incentive_prog_names.value_counts())

66

In [4]:
# Doesn't look like there's much to be done here, just going to leave it be.

incentive_progs = sorted(df.incentive_prog_names.unique())
incentive_progs

['Ajo Improvement Company',
 'Ameren',
 'Arizona Public Service',
 'Arkansas State Energy Office',
 'Austin Energy',
 'Bear Valley Electric (California Center for Sustainable Energy)',
 'CPS Energy',
 'California Energy Commission (Emerging Renewables Program)',
 'California Energy Commission (New Solar Homes Partnership)',
 'California Public Utilities Commission (California Solar Initiative)',
 'California Public Utilities Commission (Non-CSI Net Metered Systems)',
 'California Public Utilities Commission (Self-Generation Incentive Program)',
 'City of Palo Alto Utilities',
 'Connecticut Clean Energy Finance and Investment Authority',
 'Delaware Department of Natural Resources and Environmental Control',
 'Duncan Valley Electric Cooperative',
 'Efficiency Maine',
 'El Paso Electric',
 'Entergy',
 'Florida Energy & Climate Commission',
 'Gainesville Regional Utilities',
 'Graham County Electric Cooperative',
 'Illinois Department of Commerce & Economic Opportunity',
 'Imperial Irrigat

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020710 entries, 0 to 1020709
Data columns (total 33 columns):
level_0                        1020710 non-null int64
index                          1020710 non-null int64
state                          1020710 non-null object
date_installed                 1020710 non-null object
incentive_prog_names           1020710 non-null object
type                           1020710 non-null object
size_kw                        1020710 non-null float64
appraised                      1020710 non-null float64
zipcode                        1020710 non-null float64
install_type                   1020710 non-null object
installer                      1020710 non-null object
cost_per_watt                  763133 non-null float64
cost                           1020710 non-null float64
lbnl_tts                       1020710 non-null float64
city                           799148 non-null object
utility_clean                  1020710 non-null object
tech_

## Tracking + Tilt

What is the tilt value for installations that have tracking?

In [6]:
mask = df[df['tracking'] == 1]

## Starting to Drop Columns and NAs to build Final Dataset.

At this point it seems like I may have exhausted my ability to clean up this dataset much more.

columns to include:

- state                          
- date_installed                 
- incentive_prog_names           
- type                          
- size_kw                        
- appraised                      
- zipcode                        
- install_type                   
- installer                      
- cost_per_watt 
- lbnl_tts                      
- utility_clean                 
- model1_clean                   
- annual_PV_prod                 
- annual_insolation              
- tech_1                         
- rebate                         
- new_constr                     
- tracking                       
- 3rdparty                       
- bipv                           

In [7]:
df_final = df[['state', 'date_installed', 'incentive_prog_names', 'type', 'size_kw', 'appraised', 'install_type',
              'installer', 'cost_per_watt', 'lbnl_tts', 'utility_clean', 'model1_clean', 'annual_PV_prod', 
              'annual_insolation', 'tech_1', 'rebate', 'new_constr', 'tracking', '3rdparty', 'bipv']]

In [8]:
# dropping any row where we don't have cost per watt, given that's ultimately what I'm interesting in predicting.

# It might actually be more interesting to look at predicting annual PV produced, but we can look into that later.

df_final = df_final[df_final.cost_per_watt.notnull()]

In [9]:
# there are some tough calls to make here, to drop the columns with null values, or to drop a bunch of rows.
# I could try training models both ways and see what happens.

df_final.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 763133 entries, 0 to 1020709
Data columns (total 20 columns):
state                   763133 non-null object
date_installed          763133 non-null object
incentive_prog_names    763133 non-null object
type                    763133 non-null object
size_kw                 763133 non-null float64
appraised               763133 non-null float64
install_type            763133 non-null object
installer               763133 non-null object
cost_per_watt           763133 non-null float64
lbnl_tts                763133 non-null float64
utility_clean           763133 non-null object
model1_clean            422387 non-null object
annual_PV_prod          556526 non-null float64
annual_insolation       550320 non-null float64
tech_1                  422387 non-null object
rebate                  763133 non-null object
new_constr              763133 non-null float64
tracking                763133 non-null float64
3rdparty                763133 non

In [10]:
from sklearn.ensemble import RandomForestClassifier

C:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



In [11]:
rand_for = RandomForestClassifier(max_features = 'auto', min_samples_split = 20)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_final.date_installed = pd.to_datetime(df_final.date_installed)


In [14]:
df_final.date_installed = pd.to_numeric(df_final.date_installed)

In [15]:
df_final.rename(columns = {'3rdparty': "third_party"}, inplace=True)

In [16]:
import patsy

formula = "cost_per_watt ~ state + date_installed + incentive_prog_names + type + size_kw + appraised + install_type + installer + lbnl_tts + utility_clean + rebate + new_constr + tracking + third_party + bipv"

In [17]:
y, X = patsy.dmatrices(formula, df_final, return_type = 'dataframe')

MemoryError: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)